In [1]:
# coeff_coeff & coeff_mean

In [2]:
# Load
import pandas as pd

datasets = pd.read_csv("./data/pps_data/datasets.csv")

In [3]:
# PPS
def append_price_diff(dataset):
    dataset.loc[:, ["price_diff"]] = (dataset["open"] - dataset["close"]) / dataset[
        "open"
    ]
    return dataset


datasets = append_price_diff(datasets)

In [4]:
# model
import numpy as np
from sklearn.linear_model import LinearRegression


def get_linear_coef(arraylist):
    lr = LinearRegression()

    x = np.arange(len(arraylist)).reshape(-1, 1)
    y = arraylist

    lr.fit(x, y)

    linear_coef = lr.coef_
    return linear_coef[0]

In [5]:
tickers = sorted(set(datasets["ticker"]))

In [6]:
from tqdm import tqdm

In [7]:
ticker_coef_coef_dict = dict()
ticker_coef_mean_dict = dict()
for ticker in tqdm(tickers):
    dataset = datasets[datasets["ticker"] == ticker]

    coef_list = list()
    recent_n_list = [300, 200, 100, 50, 25, 10]
    for recent_n in recent_n_list:
        price_diff_cumsum_arraylist = dataset["price_diff"][-recent_n:].cumsum().values
        _coef = get_linear_coef(price_diff_cumsum_arraylist)
        coef_list.append(_coef)

    _coef_coef = get_linear_coef(coef_list)
    _coef_mean = np.average(coef_list, weights=range(1, len(recent_n_list) + 1))

    ticker_coef_coef_dict[ticker] = _coef_coef
    ticker_coef_mean_dict[ticker] = _coef_mean

100%|██████████| 1947/1947 [01:08<00:00, 28.49it/s]


In [8]:
sample_submission = pd.read_csv("./data/raw_data/sample_submission.csv")

In [13]:
sample_submission["score_coef_coef"] = sample_submission["종목코드"].map(ticker_coef_coef_dict)
sample_submission["score_coef_coef"] = sample_submission["score_coef_coef"].fillna(0)

sample_submission["score_coef_mean"] = sample_submission["종목코드"].map(ticker_coef_mean_dict)
sample_submission["score_coef_mean"] = sample_submission["score_coef_mean"].fillna(0)

sample_submission["score"] = (
    sample_submission["score_coef_coef"] + sample_submission["score_coef_mean"]
)

In [14]:
sample_submission["순위"] = (
    sample_submission["score"].rank(method="first", ascending=False).astype(int)
)
submission_result = sample_submission.loc[:, ["종목코드", "순위"]]

In [15]:
submission_result.to_csv("./data/result/linear_coeff_coeff_coef_mean_model.csv")